In [1]:
import cv2
import os
import torch, torchvision
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# !pip install git+https://github.com/davidbau/baukit

In [3]:
from baukit import show

## Preprocessing Data

In [4]:
train_df = pd.read_csv("sign_mnist_train/sign_mnist_train.csv")
test_df = pd.read_csv("sign_mnist_test/sign_mnist_test.csv")

In [5]:
def dataframe_to_nparray(train_df, test_df):
    train_df1 = train_df.copy(deep = True)
    test_df1 = test_df.copy(deep = True)
    train_images = train_df1.iloc[:, 1:].to_numpy(dtype = 'float32')
    test_images = test_df1.iloc[:, 1:].to_numpy(dtype = 'float32')
    return train_images,test_images

In [6]:
train_img, test_img = dataframe_to_nparray(train_df, test_df)
train_labels = train_df['label'].values
test_labels = test_df['label'].values

In [7]:
train_images_shaped = train_img.reshape(train_img.shape[0],1,28,28)
test_images_shaped = test_img.reshape(test_img.shape[0],1,28,28)

In [8]:
train_images_tensors = torch.from_numpy(train_images_shaped)
train_labels_tensors = torch.from_numpy(train_labels)

test_images_tensors = torch.from_numpy(test_images_shaped)
test_labels_tensors = torch.from_numpy(test_labels)

In [9]:
train_dataset = TensorDataset(train_images_tensors, train_labels_tensors) #this dataset will further devided into validation dataset and training dataset
test_dataset = TensorDataset(test_images_tensors, test_labels_tensors)

In [10]:
# transforms = transforms.Compose([
#         #transforms.Normalize(159, 40),
#         transforms.RandomHorizontalFlip(p=0.1),
#         transforms.RandomApply([transforms.RandomRotation(degrees=(-180, 180))], p=0.2),
# ]) 

In [11]:
# train_dataset = SignLanguageDataset(train_df, transform=transforms)
# test_dataset = SignLanguageDataset(test_df)

In [12]:
train_loader = DataLoader(train_dataset,
                               batch_size = 128,
                                shuffle = True,
                                num_workers=2)
test_loader = DataLoader(test_dataset,
                              batch_size = 128,
                                shuffle = True,
                                num_workers=2)

## ASL CNN MODEL

In [13]:
class ASL_Model(nn.Module):
    def __init__(self, num_classes):
        super(ASL_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(p=0.4)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv3 = nn.Conv2d(256, 512, kernel_size=3)
        self.conv4 = nn.Conv2d(512, 512, kernel_size=3)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128, 512)  # Adjust the size accordingly
        self.dropout1 = nn.Dropout(p=0.4)
        self.fc2 = nn.Linear(512, 64)
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(64, 256)
        self.dropout3 = nn.Dropout(p=0.3)
        self.fc4 = nn.Linear(256, 64)
        self.dropout4 = nn.Dropout(p=0.2)
        self.fc5 = nn.Linear(64, 256)
        self.dropout5 = nn.Dropout(p=0.3)
        self.fc6 = nn.Linear(256, 6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.dropout(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout(x)
        
        x = self.flatten(x)
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        x = F.relu(self.fc4(x))
        x = self.dropout4(x)
        x = F.relu(self.fc5(x))
        x = self.dropout5(x)
        
        x = self.fc6(x)
        return F.softmax(x, dim=num_classes)


## Loading the ASL Dataset

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# train_path = "archive/asl_alphabet_train/asl_alphabet_train"
# val_path = "aslsigndataset/splitdataset48x48/val"

In [16]:
# train_set = ImageFolder(train_path, transform = transforms.Compose([
#     transforms.Grayscale(num_output_channels=1),
#     transforms.Resize((28,28)),
# transforms.ToTensor()]))

# # val_set = ImageFolder(val_path, transform = transforms.Compose([
# #     transforms.Grayscale(num_output_channels=1),
# # #     transforms.Resize((48, 48)),
# # transforms.ToTensor()]))

# print("Number of images in the training set =", len(train_set))

In [17]:
# idx = 1500
# item = train_set[idx]
# print(f"{idx}th item is a pair", item)

In [18]:
# item[0].shape

In [19]:
# train_loader = torch.utils.data.DataLoader(
#     train_set,
#     batch_size = 128,
#     shuffle = True,
#     num_workers=2,
#     pin_memory = True
# )

# # val_loader = torch.utils.data.DataLoader(
# #     val_set,
# #     batch_size = 128,
# #     shuffle = True,
# #     num_workers=2,
# #     pin_memory = True
# # )

In [20]:
images, labels = next(train_loader.__iter__())
print(f"{images.shape=}, {labels.shape=}")

images.shape=torch.Size([128, 1, 28, 28]), labels.shape=torch.Size([128])


In [21]:
# labels

In [22]:
def train_model(model, train_loader, loss_fn, optimizer):
    model.train()
    # initiate a loss monitor
    train_loss = []
    correct_predictions = 0

    for images, labels in train_loader:
        # predict the class
        images, labels = images.to(device), labels.to(device)
        predicted = model(images)
        loss = loss_fn(predicted, labels)
        correct_predictions += (predicted.argmax(dim=1) == labels).sum().item()
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
    
    return np.mean(train_loss), correct_predictions / len(train_loader.dataset)

In [23]:
def evaluate_model(model, val_loader, loss_fn, return_confusion_matrix = False):
    model.eval()
    val_loss = []
    correct_predictions = 0
    
    if return_confusion_matrix:
        confusion_matrix = torch.zeros(
            len(val_loader.dataset.classes), len(val_loader.dataset.classes))
        
    for images, labels in val_loader:
            # predict the class
            images, labels = images.to(device), labels.to(device)
            predicted = model(images)
            loss = loss_fn(predicted, labels)
            correct_predictions += (predicted.argmax(dim=1) == labels).sum().item() 
            val_loss.append(loss.item())
            
    val_loss = np.mean(val_loss)
    val_acc = correct_predictions/ len(val_loader.dataset)
    
    if return_confusion_matrix:
        return val_loss, val_acc, confusion_matrix
    else:
        return val_loss, val_acc

In [24]:
model = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 512, kernel_size=5, stride= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.Dropout(p=0.25),
            nn.Conv2d(in_channels = 512, out_channels = 256, kernel_size=3, stride= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.Conv2d(in_channels = 256, out_channels = 128, kernel_size=3, stride= 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.Conv2d(128, 64, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),
            nn.Conv2d(64, 32, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),
            nn.Flatten(),
            nn.Linear(in_features = 4608, out_features = 512),  # Adjust the size accordingly
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(64, 26),
            nn.Softmax()
        ).to(device)

In [25]:
def view_network_parameters(model):
    tensor_list = list(model.state_dict().items())
    total_parameters = 0
    print('ModelSummary\n')
    for layer_tensor_name, ternsor in tensor_list:
        total_parameters += int(torch.numel(tensor))
        print('{}: {} elements'.format(layer_tensor_name, torch.numel(tensor)))
    print(f'\nTotal Trainable Parameters: {total_parameters})!')

def view_network_shapes(model, input_shape):
    print(summary(model, input_size = input_shape))

In [26]:
input_shape = (1,1,28,28)
view_network_shapes(model, torch.randn(input_shape).shape)

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 26]                   --
├─Conv2d: 1-1                            [1, 512, 24, 24]          13,312
├─ReLU: 1-2                              [1, 512, 24, 24]          --
├─MaxPool2d: 1-3                         [1, 512, 22, 22]          --
├─Dropout: 1-4                           [1, 512, 22, 22]          --
├─Conv2d: 1-5                            [1, 256, 20, 20]          1,179,904
├─ReLU: 1-6                              [1, 256, 20, 20]          --
├─MaxPool2d: 1-7                         [1, 256, 19, 19]          --
├─Conv2d: 1-8                            [1, 128, 17, 17]          295,040
├─ReLU: 1-9                              [1, 128, 17, 17]          --
├─MaxPool2d: 1-10                        [1, 128, 16, 16]          --
├─Conv2d: 1-11                           [1, 64, 15, 15]           32,832
├─ReLU: 1-12                             [1, 64, 15, 15]         

C:\Anaconda\Lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [27]:
# model = ASL_Model(len(train_set.classes)).to(device)
epochs = 100
learning_rate = 0.0001
weight_decay = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
training_losses = []
val_losses = []


for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_model(model, train_loader, loss_fn, optimizer)
    val_loss, val_acc = evaluate_model(model, test_loader, loss_fn)
    training_losses.append(train_loss)
    val_losses.append(val_loss)
    print(f"epoch {epoch+1}/{epochs} | train loss={np.mean(train_loss):.5f}, {train_acc=:.5f}, val loss={np.mean(val_loss):.5f}, {val_acc=:.5f}")

  1%|          | 1/100 [00:18<29:43, 18.02s/it]

epoch 1/100 | train loss=3.14861, train_acc=0.16751, val loss=3.03599, val_acc=0.28667


  2%|▏         | 2/100 [00:36<29:24, 18.00s/it]

epoch 2/100 | train loss=2.91252, train_acc=0.41231, val loss=2.92151, val_acc=0.40337


  3%|▎         | 3/100 [00:53<29:05, 17.99s/it]

...

epoch 98/100 | train loss=2.32391, train_acc=0.99898, val loss=2.38230, val_acc=0.94283


 99%|█████████▉| 99/100 [29:21<00:17, 17.79s/it]

epoch 99/100 | train loss=2.32255, train_acc=0.99989, val loss=2.38180, val_acc=0.94451


100%|██████████| 100/100 [29:39<00:00, 17.80s/it]

epoch 100/100 | train loss=2.32248, train_acc=0.99996, val loss=2.38813, val_acc=0.93600


In [35]:
model_path = os.path.join("C:/Users/Chem's bbi/Downloads/ASL-Sign-Language", "model"+".pth")
torch.save(model.state_dict(), model_path)